## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Import the random module.
import random
# Import timeit.
import timeit
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy
# Import the requests library.
import requests
# Import the time library and the datetime module from the datetime library 
import time
from datetime import datetime
import sys
import config
# Import the API key.
from config import g_key
import gmaps
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("weather_data/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,75.40,71,100,22.79,overcast clouds
1,1,Yar-Sale,RU,66.8333,70.8333,-8.81,90,44,9.48,scattered clouds
2,2,Tuktoyaktuk,CA,69.4541,-133.0374,5.00,78,20,10.36,few clouds
3,3,Carnarvon,AU,-24.8667,113.6333,71.67,88,90,8.05,overcast clouds
4,4,Port Alfred,ZA,-33.5906,26.8910,67.48,74,74,14.94,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 90


In [ ]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
city_data_df.head(10)

In [ ]:
# 4a. Determine if there are any empty rows.
#check null
city_data_df.count()


In [ ]:
city_data_df.dropna(inplace=True)
city_data_df.info

In [ ]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
#run count, if null values, use dropna()
clean_city=city_data_df.dropna()
clean_city.head()

In [ ]:
#new data frame
clean_df = city_data_df
clean_df.head(10)

In [ ]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json() 
    
    # 6f./7 Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        #hotel_df.loc[index, "Hotel Name"] = np.nan
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------") 
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [14]:
# Drop the rows where the is no Hotel Name.
hotel_df.dropna(inplace=True)
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Rikitea,PF,75.40,overcast clouds,-23.1203,-134.9692,People ThankYou
3,Carnarvon,AU,71.67,overcast clouds,-24.8667,113.6333,Hospitality Carnarvon
5,Vanimo,PG,77.76,overcast clouds,-2.6741,141.3028,Vahmoneh Lodge Vanimo
7,Yangambi,CD,71.76,overcast clouds,0.8102,24.4336,
12,Wewak,PG,77.77,overcast clouds,-3.5534,143.6268,In Wewak Boutique Hotel
14,Bengkulu,ID,74.97,scattered clouds,-3.8004,102.2655,Grage Hotel Bengkulu
16,Kapaa,US,78.78,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
25,Ocampo,MX,70.95,overcast clouds,21.6500,-101.5000,Hotel Estancia del Vaquero
28,Puerto Ayora,EC,80.56,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
30,Waingapu,ID,78.71,broken clouds,-9.6567,120.2641,Pondok Wisata Elim


In [15]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure()
fig.add_layer(marker_layer)

# 11b. Display the figure
fig



Figure(layout=FigureLayout(height='420px'))

In [20]:
# Add a heatmap of temperature for the vacation spots and a pop-up marker for each city.
## added to troubleshoot no figure/error
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))